<b>Personal Info:</b><br>
<b>Name:</b> Ehud Michel<br>
<b>ID:</b> 208952713


<b>Importing the packages:

In [2]:
# imports for reading and writing (input & output) files:
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import hebrew_tokenizer as hz
from operator import itemgetter


/Users/ehudmichel/Documents/studies/machine_learning/pythonProjectMatala


/Users/ehudmichel/opt/anaconda3/lib/python3.8/site-packages/hebrew_tokenizer/tokenizer.py:121: FutureWarning: Possible nested set at position 729
  self.scanner = re.compile(


<b>Getting the data:

In [3]:
train_filename = '.' + os.sep + 'input' + os.sep + 'annotated_corpus_for_train.xlsx'
test_filename  = '.' + os.sep + 'input' + os.sep + 'corpus_for_test.xlsx'
df_train = pd.read_excel(train_filename, 'corpus', index_col=None, na_values=['NA'])
df_test  = pd.read_excel(test_filename,  'corpus', index_col=None, na_values=['NA'])

<b>Getting the stop words:

In [4]:
with open('stop_word_list_heb_example.txt') as f:
    lines = f.readlines()[0:-1]
stop_words = []
for s in lines:
    stop_words.append(s[0:-1])


<b>function that uses the hebrew tokenizer and generates:<br> 
    1.list with the counts of the group types for each story - "num_grg_aper"<br>
    2.list with the most common tokens with thier counts of apperences - "most countes_words"<br>
    3.set with all the group types - "token_set":<br> 
        ('HEBREW','PUNC','NUMBER',etc..)<br>
    4.set with all the common types - "all_words_set"<br>

In [5]:
def get_text_idexes(story_tokens,num_of_types_per_story):   
    tokens_set = set()
    all_words_set = set()
    num_grg_aper = []
    most_countes_words = []
    for i in range(len(story_tokens)):
        number_group_occur_dict = {}
        number_words_occur_dict = {}
        for group,token,token_num,(s_i,e_i) in story_tokens[i]:
            if group in list(number_group_occur_dict.keys()):
                number_group_occur_dict[group] += 1
            else:
                number_group_occur_dict[group] = 1
            if group =='HEBREW':
                if token in list(number_words_occur_dict.keys()):
                    number_words_occur_dict[token] +=1
                else:
                    number_words_occur_dict[token] = 1
        top_words_dict = dict(sorted(number_words_occur_dict.items(),key=itemgetter(1),reverse=True)[:num_of_types_per_story])
        tokens_set.update(list(number_group_occur_dict.keys()))
        all_words_set.update(list(top_words_dict.keys()))
        num_grg_aper.append(number_group_occur_dict)
        most_countes_words.append(top_words_dict)
    return num_grg_aper, tokens_set, most_countes_words, all_words_set

<b> function to inzizlize values and new features to the manual tokenization:

In [6]:
def inizilaing_vals_in_df_manual(df_train,num_of_types_per_story):
    lang = ['HEBREW','ENGLISH']
    df_train_copy = df_train.copy() # generating copy of the dataset
    df_train_copy['tokenizer'] = df_train_copy.story.apply(lambda x: hz.tokenize(x,with_whitespaces=True)) #adding a 
    # tokeniazer column that contain tokenize object related to to story
    df_train_copy.gender = df_train_copy.gender.apply(lambda x:0 if x=='m' else 1) # modying m->0,f->1
    df_train_copy['len'] = df_train_copy['story'].apply(lambda x: len(x))#adding column name len that contains the length of each story
    num_group_count_for_story, token_type_set, most_common_words, all_words = get_text_idexes(df_train_copy['tokenizer'],num_of_types_per_story)
    #num_group_count_for_story - list of all the dictionaries that each contains counts of the toknizer groups apearences
    #token_type_set            - set with all the possible token type ('HEBREW','PANCTUAION','DATE','NUMBER','WHITE_SPACE')
    #most_common_words         - list that contains dictionary for each story with its nth(defiend in the summery function) number of most common words
    #all_words                 - set with most common words from all the stories togather

    for token in token_type_set:
        df_train_copy.insert(loc=0,column=token,value=0) #inizilizng every cell in the tokens types colmuns with 0's
    for word in all_words:
        df_train_copy.insert(loc=len(token_type_set)+2,column=word,value=0) #adding column for each word in the all words set and inizilizng every cell in every word column with 0's
    for i in range(df_train_copy.shape[0]): # for each row in the data
        for group_type in list(num_group_count_for_story[i].keys()):# for each group type in the ith story
            df_train_copy.loc[i, group_type] = num_group_count_for_story[i][group_type]
        for type_ in list(most_common_words[i].keys()): # for each type in the list[i] of dictionary keys of most common types
            df_train_copy.loc[i, type_] = most_common_words[i][type_]
    df_train_copy = df_train_copy.drop(['story','tokenizer'],axis=1)
    df_train_copy.insert(loc=len(token_type_set),column='total',value=df_train_copy[token_type_set].sum(axis=1))
    df_train_copy.insert(loc=len(token_type_set),column='amount_of_words',value=df_train_copy[lang].sum(axis=1))
    df_train_copy.insert(loc=len(token_type_set),column='len_of_paragraph',value=df_train['story'].apply(lambda x: len(x)))
    df_train_copy.insert(loc=len(token_type_set),column='len_of_paragraph_per_word',value=df_train_copy['len_of_paragraph'] / df_train_copy['amount_of_words'])
    df_train_copy.insert(loc=len(token_type_set),column='Len',value=df_train_copy['len'])
    
    df_train_copy.insert(loc=len(token_type_set),column='len_of_words',value=df_train_copy['Len'] / df_train_copy['amount_of_words'])

    # adding amount of words in story column for each story
    df_train_copy['len_of_words'] = df_train_copy['len'] / df_train_copy['amount_of_words'] # adding column with the number of tokens divide by number of types
    del df_train_copy['len']
    df_stats = df_train_copy.iloc[:,:13]
    df_manual = df_train_copy.iloc[:,14:]
    return df_stats,token_type_set,df_manual,all_words,

<b>Function to print confusion matrix and classification report:

In [7]:
def print_conMatrix(y_test,y_pred):
    print(f'Confusion Matrix:\n {confusion_matrix(y_test,y_pred)}')
      
    print (f'Accuracy:\n {round(accuracy_score(y_test,y_pred)*100, 4)}%')

    print(f'Report:\n {classification_report(y_test,y_pred)}')

<b>Function that union the stories into one list:

<b>Function that gets a set of types and return df, inizializes with 0's, of thoses types:

In [9]:
def build_df_of_types(set_of_types):   
    df_of_types = pd.DataFrame(index=df_train.index)
    for type_ in set_of_types:
        df_of_types.insert(loc=0,column=type_,value=0)
    return df_of_types

<b>Function that gets set of types and related df of the types as columns with values of 0 and 
return the df with the apperence of every type(column) in each story:

In [560]:
def set_apperences_of_types(df_of_types_0,set_of_types,df_story):
    df = df_of_types_0.copy()
    for i in range(df.shape[0]):
        for word in set_of_types:
            p = re.findall(f'(^|\s){word}\s',df_story[i])
            df.at[i,word] = len(p)
        if(i%50==0):
            print(i)
    return df

<b>Function that gets set of types and related df of the types as columns with values of 0 and 
return the df with the apperence of every type(column) in each story only once:

In [11]:
def set_if_type_in_story(df_of_types_0,set_of_types,df_story):
    df = df_of_types_0.copy()
    for i in range(df.shape[0]):
        for word in set_of_types:
            if(re.match(f'(^|\s){word}\s',df_story[i])):
                df.at[i,word] = 1
        
        if(i%50==0):
            print(i,end='\t')
    return df

Function that spilts the df to 2 seperate df's by gender:

In [12]:
def split_df_to_f_m(df_train):
    df = df_train.copy()
    df_female = df[df_train['gender']=='f']
    df_male = df[df_train['gender']=='m']
    return df_female,df_male

Function that returns the two sepreate lists of the male and female corpuses:

In [13]:
def female_male_sep_corpuses(df_male,df_female): 
    male_corpus = union_corpus(df_male.story)
    female_corpus = union_corpus(df_female.story)
    return male_corpus,female_corpus

Function that calculating dataframe with all idf words from the corpus list 

In [14]:
def idf_df(corpus,stop_words):   
    count_vectorizer = CountVectorizer(stop_words=stop_words)
    word_counts_vector = count_vectorizer.fit_transform(corpus)
    tf = pd.DataFrame(word_counts_vector.toarray(), columns=count_vectorizer.get_feature_names())
    tfidf_trans = TfidfTransformer()
    X = tfidf_trans.fit_transform(word_counts_vector)
    df = pd.DataFrame({'feature_name':count_vectorizer.get_feature_names(), 'idf_weights':tfidf_trans.idf_})
    df = df.sort_values(by=['idf_weights'],ascending=False)
    return df

<b>Function 
to make a set of all the idf words from a dataframe:

In [15]:
def df_to_set(dfx):
    top_words_a = dfx[dfx['idf_weights']>=dfx['idf_weights'][0]]
    top_words_a_set = set()
    for word in top_words_a.feature_name:
        top_words_a_set.add(word)
    return top_words_a_set

<b>Function that sums the number of types occurnces in 0/1 df:

In [16]:
def type_sum(df_01):
    df = df_01.copy()
    dict_  = {}
    for col in df.columns:
        dict_[col] = df[col].sum()
    dict_sorted = sorted(dict_.items(), key=operator.itemgetter(1),reverse=True)
    return dict_sorted

<b>Function that drop columns(types) from the 0/1 df that have less the threshold sum:

In [17]:
def drop_types(df_01,dict_sum_sorted,threshold_sum_top):
    df = df_01.copy()
    col_to_drop = []
    for i,j in dict_sum_sorted:
        if(j <=threshold_sum_top):
            col_to_drop.append(i)
    df.drop(columns=col_to_drop)
    return df

Machine learning models:

In [ ]:
def Seq_nn(input_units):
    clf = Sequential()
    clf.add(Dense(units=input_units,kernel_initializer='uniform',activation='relu'))
    clf.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))
    clf.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return clf

<b>Genretaing X and y for spilting:

In [391]:
df_female,df_male = split_df_to_f_m(df_train) #פיצול של הדאטה לפי מגדר

In [392]:
male_corpus,female_corpus = female_male_sep_corpuses(df_male,df_female)# יצירת רשימות של סיפורים לכל מגדר

In [393]:
df_female_idf = idf_df(female_corpus,stop_words) # יצירת טבלות שחיכות של מילים בקורפוס לכל מגדר בלי מילים מפסיקות בעזרת רשימת סיפורים 
df_male_idf = idf_df(male_corpus,stop_words)

In [394]:
set_female_idf = df_to_set(df_female_idf) # יצירת קבוצה של מילים שחיכות עבור כל מגדר מתוך אבלת שחיכות המילים
set_male_idf = df_to_set(df_male_idf)

In [404]:
df0_female_idf = build_df_of_types(set_female_idf)# יצירת טבלאות מאותחלות בערכים 0 עבור כל טייפ בקבוצה לפי מגדר

df0_male_idf = build_df_of_types(set_male_idf)


KeyboardInterrupt: 

In [405]:
df0_male_idf = df_male_idf[df_male_idf['idf_weights']>5.9]
df0_female_idf = df_female_idf[df_female_idf['idf_weights']>4.8]

In [407]:
df01_female_idf = set_if_type_in_story(df0_female_idf,set_female_idf,df_train.story)#יצירת טבלאות 0/1 אם הטייפ מופיע בסיפור או לא
df01_male_idf = set_if_type_in_story(df0_male_idf,set_male_idf,df_train.story)

0	50	100	150	200	250	300	350	

KeyError: 364

In [551]:
import random

In [556]:
set_female_idf_sample = random.sample(set_female_idf,2000)
set_male_idf_sample = random.sample(set_male_idf,1000)

In [557]:
set(set_female_idf_sample)
set(set_male_idf_sample)

{'בוגרי',
 'דומים',
 'בשבילנו',
 'שיתפתח',
 'כניסה',
 'הציונים',
 'המבצע',
 'מוקסמת',
 'במקדונלד',
 'ובמעגלים',
 'הזרוע',
 'רצוף',
 'התא',
 'סיפורים',
 'כאבים',
 'כואבות',
 'העברתי',
 'העקרונות',
 'נלקח',
 'הסוער',
 'המנהלת',
 'פויזציה',
 'ושלא',
 'מקובל',
 'שעושים',
 'לאיצטדיון',
 'אירועים',
 'היוודאו',
 'עמיד',
 'וחג',
 'מלעשות',
 'שטוח',
 'חגגו',
 'פרסומים',
 'יצטרף',
 'יוני',
 'השעון',
 'לחנות',
 'וחרדות',
 'מתוסכל',
 'הולדת',
 'הבורג',
 'אלתור',
 'היחיד',
 'ולהצביע',
 'ירון',
 'מהן',
 'מהבועה',
 'נפצעים',
 'חול',
 'וחזרנו',
 'שמעל',
 'תפילות',
 'לינשופית',
 'קלאס',
 'שאחריו',
 'בפנימיה',
 'פרצו',
 'המרכז',
 'בכיף',
 'מהנסיון',
 'שנפגעו',
 'עבודתם',
 'שהמנוע',
 'מאשים',
 'וקצת',
 'שמסבירות',
 'תוצאות',
 'צלם',
 'המלח',
 'דיי',
 'רדוד',
 'כשעלינו',
 'שהזעקתי',
 'לקבוצה',
 'פרנקפורט',
 'הערפל',
 'המר',
 'שבביתי',
 'הכניסה',
 'מיהרתי',
 'הרחמים',
 'הפיגו',
 'להיתקע',
 'מתנגבים',
 'כעבודה',
 'יושבי',
 'והממשלה',
 'חטוף',
 'לשירות',
 'בכללי',
 'ביד',
 'בשינוי',
 'צניחה',
 'לאימון',
 'שנ

In [561]:
df_female_idf0 = build_df_of_types(set_female_idf_sample)
df_male_idf0 = build_df_of_types(set_male_idf_sample)

In [563]:
df_female_idf01 = set_if_type_in_story(df_female_idf0,set_female_idf_sample,df_train.story)
df_male_idf01 = set_if_type_in_story(df_male_idf0,set_male_idf_sample,df_train.story)

0	50	100	150	200	250	300	350	0	50	100	150	200	250	300	350	

In [625]:
df_stats_300,stats_colmuns_300,df_manual_300,all_wrds_set_300 = df_stats,stats_colmuns,df_manual,all_wrds_set 

In [736]:
df_stats_250,stats_colmuns_250,df_manual_250,all_wrds_set_250 = df_stats,stats_colmuns,df_manual,all_wrds_set 

In [862]:
df_stats_175,stats_colmuns_175,df_manual_175,all_wrds_set_175 = df_stats,stats_colmuns,df_manual,all_wrds_set 

In [867]:
df_stats,stats_colmuns,df_manual,all_wrds_set = inizilaing_vals_in_df_manual(df_train,150)
# עיבוד מידע על הדאטה בצורה ידנית

In [902]:
dict_df_manual = type_sum(df_manual)

df_manual_droped = df_manual.copy()
col_to_drop = []
for i,j in dict_df_manual:
    if(j <=13):
        col_to_drop.append(i)
df_manual_droped = df_manual_droped.drop(columns=col_to_drop)
df_manual_droped = df_manual_droped.copy()
col_to_drop = []
for i,j in dict_df_manual:
    if(j >=55):
        col_to_drop.append(i)
df_manual_droped = df_manual_droped.drop(columns=col_to_drop)
df_merge = pd.concat([df_stats,df_manual_droped],axis=1)
X = df_merge
y = df_train.gender
y = y.apply(lambda x:0 if x=='m' else 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
clf_LG = LogisticRegression(random_state=0,max_iter=10000000).fit(X_train, y_train)
y_pred_LG = clf_LG.predict(X_test)
print_conMatrix(y_test=y_test,y_pred=y_pred_LG)

Confusion Matrix:
 [[52  2]
 [13  6]]
Accuracy:
 79.4521%
Report:
               precision    recall  f1-score   support

           0       0.80      0.96      0.87        54
           1       0.75      0.32      0.44        19

    accuracy                           0.79        73
   macro avg       0.78      0.64      0.66        73
weighted avg       0.79      0.79      0.76        73



In [926]:
max_ = 0
bz = 5
by = 50
for z in range(5,100):
    for w in range(50,300):
        dict_df_manual = type_sum(df_manual)
        df_manual_droped = df_manual.copy()
        col_to_drop = []
        for i,j in dict_df_manual:
            if(j <=13):
                col_to_drop.append(i)
        df_manual_droped = df_manual_droped.drop(columns=col_to_drop)
        df_manual_droped = df_manual_droped.copy()
        col_to_drop = []
        for i,j in dict_df_manual:
            if(j >=55):
                col_to_drop.append(i)
        df_manual_droped = df_manual_droped.drop(columns=col_to_drop)
        df_merge = pd.concat([df_stats,df_manual_droped],axis=1)
        X = df_merge
        y = df_train.gender
        y = y.apply(lambda x:0 if x=='m' else 1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        clf_LG = LogisticRegression(random_state=0,max_iter=10000000).fit(X_train, y_train)
        y_pred_LG = clf_LG.predict(X_test)

        f1 = f1_score(y_test,y_pred_LG ,average='macro')
        if(float(f1)> max_):
            max_ = f1
            bz = z
            by = w
    print(max_)
print(f"the best f1 is:{max_},in index[{bz}:{by}]")


KeyboardInterrupt: 

In [910]:
f1_score(y_test,y_pred_LG ,average='macro')

AttributeError: 'numpy.float64' object has no attribute 'type'

In [898]:
from sklearn.metrics import f1_score

In [825]:
dict_df_manual = type_sum(df_manual)

df_manual_droped = df_manual.copy()
col_to_drop = []
for i,j in dict_df_manual:
    if(j <=0):
        col_to_drop.append(i)
df_manual_droped = df_manual_droped.drop(columns=col_to_drop)
df_manual_droped = df_manual_droped.copy()
col_to_drop = []
for i,j in dict_df_manual:
    if(j >=900):
        col_to_drop.append(i)
df_manual_droped = df_manual_droped.drop(columns=col_to_drop)
df_merge = pd.concat([df_stats,df_manual_droped],axis=1)
X = df_merge
y = df_train.gender
y = y.apply(lambda x:0 if x=='m' else 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
gnb = GaussianNB()
y_pred_gnb = gnb.fit(X_train, y_train).predict(X_test)
print_conMatrix(y_test=y_test,y_pred=y_pred_LG)

Confusion Matrix:
 [[52  2]
 [11  8]]
Accuracy:
 82.1918%
Report:
               precision    recall  f1-score   support

           0       0.83      0.96      0.89        54
           1       0.80      0.42      0.55        19

    accuracy                           0.82        73
   macro avg       0.81      0.69      0.72        73
weighted avg       0.82      0.82      0.80        73

